# SuStaIn GPU Benchmark
**Instructions:** Runtime → Change runtime type → T4 GPU

In [ ]:
!pip install -q numpy scipy matplotlib scikit-learn pandas tqdm pathos

import os
if os.path.exists('mphil'):
    %cd mphil
    !git pull origin main
else:
    !git clone https://github.com/Amelia3141/mphil.git
    %cd mphil

import torch
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
import numpy as np
import time, os, shutil, sys
sys.path.insert(0, 'pySuStaIn')

def generate_test_data(n_patients, n_biomarkers, seed=42):
    np.random.seed(seed)
    data = np.random.randint(0, 4, (n_patients, n_biomarkers))
    prob_nl = np.where(data == 0, 0.85, 0.0375)
    prob_score = np.zeros((n_patients, n_biomarkers, 3))
    for b in range(n_biomarkers):
        for z in range(3):
            prob_score[:, b, z] = np.where(data[:, b] == z+1, 0.85, 0.0375)
    return prob_nl, prob_score, np.tile(np.arange(1,4), (n_biomarkers,1)), [f'B{i}' for i in range(n_biomarkers)]

def benchmark(n_patients, n_biomarkers, use_gpu):
    from pySuStaIn import OrdinalSustain
    from pySuStaIn.TorchOrdinalSustain import TorchOrdinalSustain
    prob_nl, prob_score, score_vals, labels = generate_test_data(n_patients, n_biomarkers)
    folder = f'./bench_{"gpu" if use_gpu else "cpu"}'
    if os.path.exists(folder): shutil.rmtree(folder)
    os.makedirs(folder)
    if use_gpu:
        model = TorchOrdinalSustain(prob_nl, prob_score, score_vals, labels, 5, 1, 2000, folder, 'b', True, 42, use_gpu=True)
    else:
        model = OrdinalSustain(prob_nl, prob_score, score_vals, labels, 5, 1, 2000, folder, 'b', False, 42)
    start = time.time()
    model.run_sustain_algorithm(plot=False)
    return time.time() - start

print('Setup complete')

In [ ]:
results = []
for n in [200, 500, 1000]:
    print(f'\n=== {n} patients, 15 biomarkers ===')
    t_cpu = benchmark(n, 15, False)
    print(f'CPU: {t_cpu:.1f}s')
    t_gpu = benchmark(n, 15, True)
    print(f'GPU: {t_gpu:.1f}s')
    speedup = t_cpu/t_gpu
    print(f'Speedup: {speedup:.2f}x')
    results.append({'n_patients': n, 'cpu_sec': t_cpu, 'gpu_sec': t_gpu, 'speedup': speedup})

import pandas as pd
df = pd.DataFrame(results)
print('\n' + '='*50)
print('BENCHMARK SUMMARY')
print('='*50)
print(df.to_string(index=False))
df.to_csv('benchmark_results.csv', index=False)